In [1]:
import run_simulation

In [2]:
run_simulation.main()

PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_fb813911-b203-40e2-be30-cd4afcc8650b.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 63843.
PyAEDT ERROR: Error getting status: Cannot connect to license server system. (-15,10:10061 "WinSock: Connection refused")
PyAEDT WARNING: Electronics Desktop license not found on the default license server.
PyAEDT INFO: Electronics Desktop started on gRPC port: 63843 after 9.116193532943726 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.Elec

In [ ]:
import run_simulation

# 1. Simulation 클래스의 인스턴스를 생성합니다.
simulation_runner = run_simulation.Simulation()

# 2. create_design 메서드를 호출하여 프로젝트와 디자인을 초기화합니다.
# 이 과정에서 내부에 self.design 객체가 생성됩니다.
simulation_runner.create_design("SST_MFT")

# 3. 이제 입력 매개변수를 생성할 수 있습니다.
# 이 메서드는 초기화된 design 객체를 사용하게 됩니다.
input_parameters = simulation_runner.create_input_parameter()

# 4. 생성된 파라미터를 Simulation 객체와 Ansys 디자인에 설정합니다.
simulation_runner.set_variable(input_parameters)

# 5. 해석 설정 및 실행
simulation_runner.set_maxwell_analysis()

# 6. 모델을 생성합니다.
simulation_runner.create_core()
simulation_runner.create_face(simulation_runner.maxwell_design)
simulation_runner.create_windings()
simulation_runner.create_mold()
simulation_runner.create_cold_plate()
simulation_runner.create_air()

# 7. 메쉬 및 경계조건 설정
simulation_runner.assign_meshing()
simulation_runner.assign_excitations()

# 8. 해석 설정 및 실행
simulation_runner.analyze_maxwell()

# 9. 결과 리포팅
simulation_runner.get_simulation_results(input=True)

# 10. 두 번째 해석 실행
simulation_runner.second_simulation()

# 11. Icepak 디자인 생성
simulation_runner.create_icepak()       

# 12. Icepak 해석 설정
simulation_runner.setup_icepak_analysis()

# 13. Icepak 해석 실행
simulation_runner.analyze_icepak()

# 14. Icepak 결과 리포팅
simulation_runner.get_icepak_results()

# 15. 결과 저장
simulation_runner.save_results_to_csv(simulation_runner.results_df)




PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_e43a74c8-c500-4a9d-928a-9e1760c6709d.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 56828.
PyAEDT ERROR: Error getting status: Cannot connect to license server system. (-15,10:10061 "WinSock: Connection refused")
PyAEDT WARNING: Electronics Desktop license not found on the default license server.
PyAEDT INFO: Electronics Desktop started on gRPC port: 56828 after 5.053632736206055 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.Elec

In [5]:
simulation_runner.maxwell_design.leg_left.name

icepak = simulation_runner.maxwell_design.get_active_design()

#icepak._get_module()
i_cold_plate_top = simulation_runner.icepak_design.model3d.find_object(simulation_runner.maxwell_design.cold_plate_top)
i_cold_plate_bottom = simulation_runner.icepak_design.model3d.find_object(simulation_runner.maxwell_design.cold_plate_bottom)
assignment=[i_cold_plate_top, i_cold_plate_bottom]

object_name = [obj if isinstance(obj, str) else obj.name for obj in assignment]

object_name



icepak.assign_source





PyAEDT INFO: Active Design set to IcepakDesign1
PyAEDT INFO: Active Design set to IcepakDesign1
PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Returning found Desktop session with PID 22680!
PyAEDT INFO: No project is defined. Project simulation61 exists and has been read.
PyAEDT INFO: Active Design set to IcepakDesign1
PyAEDT INFO: Aedt Objects correctly read


<bound method Icepak.assign_source of <pyaedt_module.solver.icepak.Icepak object at 0x000001A784985850>>

In [5]:
simulation_runner.w1

np.float64(223.4)

In [ ]:
simulation_runner.winding1.faces

In [ ]:
input_parameter = sim1.create_input_parameter()

sim1.set_variable(input_parameter)


In [ ]:
sim1.create_core()


In [6]:
def offset_calculation(coil_diameter, h1, height_ratio, offset_z) :

    plus = coil_diameter/2
    minus = ((h1*height_ratio) - coil_diameter/2)

    offset = (plus + minus)/2 - plus + offset_z
    offset = (plus + minus)/2 - plus

    return offset

offset1 = offset_calculation(sim1.N1_coil_diameter, sim1.h1, sim1.N1_height_ratio, sim1.N1_offset)
winding_params1 = {
    "N" : sim1.N1,
    "N_layer" : sim1.N1_layer,
    "x" : "w1 + 2*N1_space_w",
    "y" : "l1 + 2*N1_space_l",
    "coil_diameter" : "N1_coil_diameter",
    "coil_zgap" : "N1_coil_zgap",
    "coil_layer_x_gap" : "N1_layer_gap",
    "coil_layer_y_gap" : "N1_layer_gap",
    "color" : [255, 50, 50],
    "transparency" : 0,
    "offset" : ["0mm", "-(l1+l2)/2", f"{offset1}mm"],
    "terminal_position" : "w1/2 + 150mm",
    "Num" : 18
}
sim1.create_winding(name="winding1", **winding_params1)

offset2 = offset_calculation(sim1.N2_coil_diameter, sim1.h1, sim1.N2_height_ratio, sim1.N2_offset)
winding_params2 = {
    "N" : sim1.N2,
    "N_layer" : sim1.N2_layer,
    "x" : "w1 + 2*N2_space_w",
    "y" : "l1 + 2*N2_space_l",
    "coil_diameter" : "N2_coil_diameter",
    "coil_zgap" : "N2_coil_zgap",
    "coil_layer_x_gap" : "N2_layer_gap",
    "coil_layer_y_gap" : "N2_layer_gap",
    "color" : [50, 50, 255],
    "transparency" : 0,
    "offset" : ["0mm", "(l1+l2)/2", f"{offset2}mm"],
    "terminal_position" : "w1/2 + 150mm",
    "Num" : 18
}
sim1.create_winding(name="winding2", **winding_params2)